In [2]:
%load_ext autoreload
%autoreload 2

## Test: a simple model

In [3]:
from hackaton_eet.data.loaders import get_data
from hackaton_eet.data.transformers import align_data

First, let's get the data. To maket his easier, a helper function `get_data` has been created. As inputs, this function expects a dictionary of your desired datasets, their columns and lags. Note that all lags are given in terms of 15 minutes, and the dates on the column are the day you are trying to predict.

Since we run the model each day at 10:00:00, this means that lag 1 is 09:45:00, and so on.

The code knows of each set which bits are known in advance and which parts are only known after the fact, and will only return the data you have available at each time. It is highly recommended to use this function instead of writing your own.

In [182]:
feature_datasets = {
#     'tennet.verrekenprijzen': {
#         'lags': [1, 6, 13, 15],  # Lags are now days. Makes working with them easier
#         'columns': ['invoeden', 'Afnemen']
#     },
    'pricehub.pricehub': {
        'lags': [1, ],
        'columns': ['EEXSPOT', 'PNSPOT', 'APXSPOT', 'BPXSPOT',
                    'EEXCHSPOT', 'N2EXSPOT', 'TTF', 'EUA',
                    'SPARK_NL', 'APIEUR', 'DARK_DE', 'SPARK_NL'],
    }
}
target_dataset = {
    'tennet.target': {}
}

feature_data = get_data(feature_datasets)
target_data = get_data(target_dataset)

ValueError: cannot reshape array of size 730570 into shape (11)

In [ ]:
feature_data

## Training the model

In [164]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, explained_variance_score, mean_squared_error

In [165]:
s = 24*4*400

In [166]:
X = feature_data.fillna(0).values
y = target_data.fillna(0).values

X, y = align_data(X, y)

X, y = X.T, y.T

X_train, X_test = X[:s], X[s:]
y_train, y_test = y[:s], y[s:]

In [167]:
linest = linear_model.LinearRegression()
linest = linest.fit(X_train, y_train)
y_pred = linest.predict(X_test)

In [168]:
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print("VAF: %.2f" % explained_variance_score(y_test, y_pred))

MAE: 27.71
VAF: 0.02


In [116]:
linest

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [117]:
est = RandomForestRegressor(
    n_estimators=200,
    max_depth=3,
    n_jobs=-1,
)
est = est.fit(X_train, y_train)
y_pred = est.predict(X_test)

So, did it work?

In [118]:
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print("VAF: %.2f" % explained_variance_score(y_test, y_pred))

MAE: 29.09
VAF: 0.02


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(y_pred[:, 1] - y_test[:, 1])